# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
from os import getcwd
#Getting the dataset from the drive
path_dataset = f"{getcwd()}/../content/drive/My Drive/upgrad_datasets/Project_data.zip"

#UnZipping the File
from zipfile import ZipFile
file_name = path_dataset

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [3]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2 MB 53 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
jax 0.2.21 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

In [6]:
import tensorflow as tf

In [7]:
'''
%tensorflow_version 1.5.0
import tensorflow.compat.v1 as tf
print(tf.__version__)
'''

'\n%tensorflow_version 1.5.0\nimport tensorflow.compat.v1 as tf\nprint(tf.__version__)\n'

We set the random seed so that the results don't vary drastically.

In [8]:
import numpy as np
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [9]:
train_doc = np.random.permutation(open('/content/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/Project_data/val.csv').readlines())
batch_size = 32

In [10]:
#Resize for 3D models
def crop_resize_img(img):
  if img.shape[1]==160:
    img = img[:120, 20:140]
  resized_image = imresize(img,(120,120,3)).astype(np.float32) #for conv3D
  #resized_image = imresize(img,(120,120)).astype(np.float32)  #for conv2D
  return resized_image

In [11]:
y = 120   
z = 120   # (y,z) is the final size of the input images 
channel = 3  # number of 
channels in images 3 for color(RGB)
x=18         # x is the number of images you use for each video

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [12]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.round(np.linspace(0,29,18)).astype(int) #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    crop_resize_image = crop_resize_img(image)
                    
                    batch_data[folder,idx,:,:,0] = crop_resize_image[:, : , 0]/255    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = crop_resize_image[:, : , 1]/255   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = crop_resize_image[:, : , 2]/255   #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size != 0):
          batch_size = len(t)%batch_size
          batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
          batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
          for folder in range(batch_size): # iterate over the batch_size
              imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
              for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                      
                      #crop the images and resize them. Note that the images are of 2 different shape 
                      #and the conv3D will throw error if the inputs in a batch have different shapes
                  crop_resize_image = crop_resize_img(image)
                      
                  batch_data[folder,idx,:,:,0] = crop_resize_image[:, : , 0]/255   #normalise and feed in the image
                  batch_data[folder,idx,:,:,1] = crop_resize_image[:, : , 1]/255   #normalise and feed in the image
                  batch_data[folder,idx,:,:,2] = crop_resize_image[:, : , 2]/255   #normalise and feed in the image

              batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
          yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember
                    
                   

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [13]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/Project_data/train'
val_path = '/content/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model Training
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

##Model 1
####Conv3D with Batch size = 32 
####No. of epochs = 20

In [14]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, Conv2D, MaxPooling3D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In [15]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [16]:
#write your model here
nb_classes = 5
# Input
input_shape=(x,y,z,channel)

In [ ]:
model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(32,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(16,activation='relu'))
model.add(BatchNormalization())


model.add(Dense(nb_classes,activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
#from tf.keras.optimizers import Adam
optimiser = tf.keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.99) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_16 (Conv3D)           (None, 18, 120, 120, 16)  1312      
_________________________________________________________________
activation_16 (Activation)   (None, 18, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 18, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 9, 60, 60, 16)     0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 9, 60, 60, 32)     4128      
_________________________________________________________________
activation_17 (Activation)   (None, 9, 60, 60, 32)     0         
_________________________________________________________________
batch_normalization_29 (Batc (None, 9, 60, 60, 32)    

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
model_name = 'model_gesture' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [21]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 32
Epoch 1/20
21/21 [==============================] - 55s 3s/step - loss: 1.5868 - categorical_accuracy: 0.3695 - val_loss: 1.8403 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_gesture_2021-09-2914_04_41.761424/model-00001-1.58676-0.36953-1.84029-0.20000.h5
Epoch 2/20
21/21 [==============================] - 35s 2s/step - loss: 1.3154 - categorical_accuracy: 0.4948 - val_loss: 1.7888 - val_categorical_accuracy: 0.1875

Epoch 00002: saving model to model_gesture_2021-09-2914_04_41.761424/model-00002-1.31536-0.49482-1.78885-0.18750.h5
Epoch 3/20
21/21 [==============================] - 31s 2s/step - loss: 1.2097 - categorical_accuracy: 0.5504 - val_loss: 2.2347 - val_categorical_accuracy: 0.1875

Epoch 00003: saving model to model_gesture_2021-09-2914_04_41.761424/model-00003-1.20970-0.55035-2.23470-0.18750.h5
Epoch 4/20
21/21 [==============================] - 31s 2s/step - loss: 1.1060 - categorical_acc

###Deleting saved model folders as accuracy of train and validation data is not significant enough.

In [ ]:
import shutil
shutil.rmtree('/content/model_gesture_2021-09-2913_40_35.454796')
shutil.rmtree('/content/model_gesture_2021-09-2913_53_42.575123')
shutil.rmtree('/content/model_gesture_2021-09-2914_04_41.761424')

##Model 2
Adding more layers, increasing the dense neurons and drop out

In [ ]:
nb_classes = 5
input_shape=(x,y,z,channel)

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(16, (3,3,3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(32, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(64, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(128, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(nb_classes,activation='softmax'))

#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_20 (Conv3D)           (None, 18, 120, 120, 16)  1312      
_________________________________________________________________
activation_20 (Activation)   (None, 18, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_35 (Batc (None, 18, 120, 120, 16)  64        
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 18, 120, 120, 16)  6928      
_________________________________________________________________
activation_21 (Activation)   (None, 18, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_36 (Batc (None, 18, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 9, 60, 60, 16)    

In [ ]:
model_name = 'model_gesture' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
21/21 [==============================] - 40s 2s/step - loss: 1.8687 - categorical_accuracy: 0.3810 - val_loss: 5.5948 - val_categorical_accuracy: 0.1875

Epoch 00001: saving model to model_gesture_2021-09-2914_04_41.761424/model-00001-1.86865-0.38095-5.59483-0.18750.h5
Epoch 2/20
21/21 [==============================] - 33s 2s/step - loss: 1.5003 - categorical_accuracy: 0.4566 - val_loss: 3.8174 - val_categorical_accuracy: 0.1875

Epoch 00002: saving model to model_gesture_2021-09-2914_04_41.761424/model-00002-1.50028-0.45658-3.81742-0.18750.h5
Epoch 3/20
21/21 [==============================] - 34s 2s/step - loss: 1.3576 - categorical_accuracy: 0.5070 - val_loss: 2.1419 - val_categorical_accuracy: 0.3750

Epoch 00003: saving model to model_gesture_2021-09-2914_04_41.761424/model-00003-1.35759-0.50700-2.14193-0.37500.h5
Epoch 4/20
21/21 [==============================] - 32s 2s/step - loss: 1.2586 - categorical_accuracy: 0.5462 - val_loss: 2.6239 - val_categorical_accuracy: 

In [ ]:
shutil.rmtree('/content/model_gesture_2021-09-2906_34_24.005771')

##Model 3 
Using Stochastic Gradient Descent as Optimizer

Number of Epochs = 30

In [ ]:
nb_classes = 5
input_shape=(x,y,z,channel)

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(16, (3,3,3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(32, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(64, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(128, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization(momentum=0.8))
model.add(Dropout(0.3))


model.add(Dense(256,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(BatchNormalization(momentum=0.8))
model.add(Dropout(0.3))

model.add(Dense(nb_classes,activation='softmax'))

#Optimizer
optimiser = tf.keras.optimizers.SGD() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_52 (Conv3D)           (None, 18, 120, 120, 16)  1312      
_________________________________________________________________
activation_52 (Activation)   (None, 18, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_71 (Batc (None, 18, 120, 120, 16)  64        
_________________________________________________________________
conv3d_53 (Conv3D)           (None, 18, 120, 120, 16)  6928      
_________________________________________________________________
activation_53 (Activation)   (None, 18, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_72 (Batc (None, 18, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_30 (MaxPooling (None, 9, 60, 60, 16)    

In [ ]:
model_name = 'model_gesture' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 36s 2s/step - loss: 2.0785 - categorical_accuracy: 0.3333 - val_loss: 41.2503 - val_categorical_accuracy: 0.1875

Epoch 00001: saving model to model_gesture_2021-09-2914_31_59.720448/model-00001-2.07849-0.33333-41.25032-0.18750.h5
Epoch 2/30
21/21 [==============================] - 33s 2s/step - loss: 1.5873 - categorical_accuracy: 0.4538 - val_loss: 7.4338 - val_categorical_accuracy: 0.1875

Epoch 00002: saving model to model_gesture_2021-09-2914_31_59.720448/model-00002-1.58727-0.45378-7.43377-0.18750.h5
Epoch 3/30
21/21 [==============================] - 34s 2s/step - loss: 1.5211 - categorical_accuracy: 0.4902 - val_loss: 3.1562 - val_categorical_accuracy: 0.3125

Epoch 00003: saving model to model_gesture_2021-09-2914_31_59.720448/model-00003-1.52109-0.49020-3.15622-0.31250.h5
Epoch 4/30
21/21 [==============================] - 34s 2s/step - loss: 1.3746 - categorical_accuracy: 0.5014 - val_loss: 23.6284 - val_categorical_accurac

In [ ]:
shutil.rmtree('/content/model_gesture_2021-09-2914_31_59.720448')

##Model 4 
CNN-LSTM Model

In [17]:
from keras.layers.recurrent import LSTM
input_shape=(x,y,z,channel)
lstm_cells=64
dense_neurons=256
dropout=0.3
nb_classes = 5

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(512, (2, 2) , padding='valid', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(lstm_cells))
model.add(Dropout(dropout))

model.add(Dense(dense_neurons,activation='relu',kernel_regularizer = tf.keras.regularizers.l2(0.0001)))
model.add(Dropout(dropout))

model.add(Dense(nb_classes, activation='softmax'))
optimiser = tf.keras.optimizers.SGD()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [18]:
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 18, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 18, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 18, 30, 30, 64)    1

In [19]:
model_name = 'model_gesture' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [22]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - 99s 3s/step - loss: 1.5859 - categorical_accuracy: 0.2685 - val_loss: 1.5484 - val_categorical_accuracy: 0.3300

Epoch 00001: saving model to model_gesture_2021-09-2915_48_41.305220/model-00001-1.58586-0.26848-1.54843-0.33000.h5
Epoch 2/30
21/21 [==============================] - 38s 2s/step - loss: 1.4893 - categorical_accuracy: 0.3313 - val_loss: 1.4374 - val_categorical_accuracy: 0.3125

Epoch 00002: saving model to model_gesture_2021-09-2915_48_41.305220/model-00002-1.48926-0.33126-1.43736-0.31250.h5
Epoch 3/30
21/21 [==============================] - 33s 2s/step - loss: 1.4411 - categorical_accuracy: 0.3841 - val_loss: 1.4776 - val_categorical_accuracy: 0.1875

Epoch 00003: saving model to model_gesture_2021-09-2915_48_41.305220/model-00003-1.44109-0.38407-1.47764-0.18750.h5
Epoch 4/30
21/21 [==============================] - 31s 2s/step - loss: 1.3897 - categorical_acc

In [ ]:
shutil.rmtree('/content/model_gesture_2021-09-2906_34_24.005771')

##Model 5
CNN+LSTM but reducing the parameters

In [36]:
from keras.layers.recurrent import LSTM
input_shape=(x,y,z,channel)
lstm_cells=32
dense_neurons=64
dropout=0.3
nb_classes = 5

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(512, (2, 2) , padding='valid', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(lstm_cells))
model.add(Dropout(dropout))

model.add(Dense(dense_neurons,activation='relu',kernel_regularizer = tf.keras.regularizers.l2(0.0001)))
model.add(Dropout(dropout))

model.add(Dense(nb_classes, activation='softmax'))
optimiser = tf.keras.optimizers.SGD(learning_rate=0.005)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [37]:
print (model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_108 (TimeDi (None, 18, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_109 (TimeDi (None, 18, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_110 (TimeDi (None, 18, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_111 (TimeDi (None, 18, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_112 (TimeDi (None, 18, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_113 (TimeDi (None, 18, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_114 (TimeDi (None, 18, 30, 30, 64)   

In [38]:
model_name = 'model_gesture' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [39]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 35s 2s/step - loss: 1.6076 - categorical_accuracy: 0.2017 - val_loss: 1.6285 - val_categorical_accuracy: 0.1250

Epoch 00001: saving model to model_gesture_2021-09-2915_48_41.305220/model-00001-1.60758-0.20168-1.62850-0.12500.h5
Epoch 2/30
21/21 [==============================] - 29s 1s/step - loss: 1.5645 - categorical_accuracy: 0.2941 - val_loss: 1.5813 - val_categorical_accuracy: 0.3125

Epoch 00002: saving model to model_gesture_2021-09-2915_48_41.305220/model-00002-1.56446-0.29412-1.58128-0.31250.h5
Epoch 3/30
21/21 [==============================] - 28s 1s/step - loss: 1.5483 - categorical_accuracy: 0.3053 - val_loss: 1.5120 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_gesture_2021-09-2915_48_41.305220/model-00003-1.54827-0.30532-1.51195-0.25000.h5
Epoch 4/30
21/21 [==============================] - 28s 1s/step - loss: 1.5015 - categorical_accuracy: 0.3249 - val_loss: 1.4167 - val_categorical_accuracy: 

In [ ]:
shutil.rmtree('/content/model_gesture_2021-09-2906_34_24.005771')

##Model 6
GRU with CNN

In [40]:
input_shape=(x,y,z,channel)
lstm_cells=64
dense_neurons=256
dropout=0.3
nb_classes = 5
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(512, (2, 2) , padding='valid', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))


model.add(GRU(lstm_cells))
model.add(Dropout(dropout))

model.add(Dense(dense_neurons,activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.0001)))
model.add(Dropout(dropout))

model.add(Dense(nb_classes, activation='softmax'))
optimiser = tf.keras.optimizers.Adam(lr=0.0002)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_127 (TimeDi (None, 18, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_128 (TimeDi (None, 18, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_129 (TimeDi (None, 18, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_130 (TimeDi (None, 18, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_131 (TimeDi (None, 18, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_132 (TimeDi (None, 18, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_133 (TimeDi (None, 18, 30, 30, 64)   

In [41]:
model_name = 'model_gesture' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [42]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 33s 1s/step - loss: 1.5268 - categorical_accuracy: 0.3137 - val_loss: 1.5955 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_gesture_2021-09-2915_48_41.305220/model-00001-1.52680-0.31373-1.59551-0.25000.h5
Epoch 2/30
21/21 [==============================] - 26s 1s/step - loss: 1.3001 - categorical_accuracy: 0.4230 - val_loss: 0.9869 - val_categorical_accuracy: 0.6875

Epoch 00002: saving model to model_gesture_2021-09-2915_48_41.305220/model-00002-1.30012-0.42297-0.98688-0.68750.h5
Epoch 3/30
21/21 [==============================] - 27s 1s/step - loss: 1.0522 - categorical_accuracy: 0.5938 - val_loss: 1.0562 - val_categorical_accuracy: 0.5625

Epoch 00003: saving model to model_gesture_2021-09-2915_48_41.305220/model-00003-1.05223-0.59384-1.05624-0.56250.h5
Epoch 4/30
21/21 [==============================] - 26s 1s/step - loss: 0.9809 - categorical_accuracy: 0.6527 - val_loss: 1.0840 - val_categorical_accuracy: 

In [ ]:
shutil.rmtree('/content/model_gesture_2021-09-2906_34_24.005771')

##Model 7
GRU with CNN and Dropout increased from 0.3 to 0.5

In [43]:
input_shape=(x,y,z,channel)
lstm_cells=64
dense_neurons=256
dropout=0.5
nb_classes = 5
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(512, (2, 2) , padding='valid', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))


model.add(GRU(lstm_cells))
model.add(Dropout(dropout))

model.add(Dense(dense_neurons,activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.0001)))
model.add(Dropout(dropout))

model.add(Dense(nb_classes, activation='softmax'))
optimiser = tf.keras.optimizers.Adam(lr=0.0002)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_146 (TimeDi (None, 18, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_147 (TimeDi (None, 18, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_148 (TimeDi (None, 18, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_149 (TimeDi (None, 18, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_150 (TimeDi (None, 18, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_151 (TimeDi (None, 18, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_152 (TimeDi (None, 18, 30, 30, 64)   

In [44]:
model_name = 'model_gesture' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [45]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 30s 1s/step - loss: 1.7378 - categorical_accuracy: 0.2577 - val_loss: 1.5459 - val_categorical_accuracy: 0.3125

Epoch 00001: saving model to model_gesture_2021-09-2915_48_41.305220/model-00001-1.73779-0.25770-1.54589-0.31250.h5
Epoch 2/30
21/21 [==============================] - 27s 1s/step - loss: 1.5781 - categorical_accuracy: 0.3081 - val_loss: 1.2498 - val_categorical_accuracy: 0.5625

Epoch 00002: saving model to model_gesture_2021-09-2915_48_41.305220/model-00002-1.57812-0.30812-1.24985-0.56250.h5
Epoch 3/30
21/21 [==============================] - 26s 1s/step - loss: 1.4277 - categorical_accuracy: 0.4090 - val_loss: 1.2174 - val_categorical_accuracy: 0.6250

Epoch 00003: saving model to model_gesture_2021-09-2915_48_41.305220/model-00003-1.42765-0.40896-1.21739-0.62500.h5
Epoch 4/30
21/21 [==============================] - 28s 1s/step - loss: 1.2850 - categorical_accuracy: 0.4706 - val_loss: 1.2289 - val_categorical_accuracy: 

#Transfer Learning

##Model 8
Transfer Learning with MobileNet

In [46]:
from keras.applications import mobilenet

In [53]:
mobilenet_transfer = mobilenet.MobileNet(weights = 'imagenet', include_top = False)

input_shape=(x,y,z,channel)
gru_cells=64
dense_neurons=64
dropout=0.3
nb_classes = 5

model = Sequential()
model.add(TimeDistributed(mobilenet_transfer,input_shape=input_shape))

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2), padding='same')))

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2), padding='same')))

model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2), padding='same')))

model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))
model.add(TimeDistributed(MaxPooling2D((2, 2), padding='same')))

model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization(momentum=0.8)))

model.add(TimeDistributed(Flatten()))


model.add(GRU(lstm_cells))
model.add(Dropout(dropout))

model.add(Dense(dense_neurons,activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.0001)))
model.add(Dropout(dropout))

model.add(Dense(nb_classes, activation='softmax'))
optimiser = tf.keras.optimizers.Adam(lr=0.0002)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_247 (TimeDi (None, 18, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_248 (TimeDi (None, 18, 3, 3, 16)      147472    
_________________________________________________________________
time_distributed_249 (TimeDi (None, 18, 3, 3, 16)      64        
_________________________________________________________________
time_distributed_250 (TimeDi (None, 18, 2, 2, 16)      0         
_________________________________________________________________
time_distributed_251 (TimeDi (None, 18, 2, 2, 32)      4640      
_________________________________________________________________
time_distributed_252 (TimeDi (None, 18, 2, 2, 32)      128       
_________________________________________________________________
time_distributed_253 (TimeDi (None, 18, 1, 1, 32)    

In [54]:
model_name = 'model_gesture' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [55]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 43s 2s/step - loss: 1.7278 - categorical_accuracy: 0.1961 - val_loss: 1.6742 - val_categorical_accuracy: 0.1250

Epoch 00001: saving model to model_gesture_2021-09-2915_48_41.305220/model-00001-1.72781-0.19608-1.67416-0.12500.h5
Epoch 2/30
21/21 [==============================] - 30s 1s/step - loss: 1.6074 - categorical_accuracy: 0.2773 - val_loss: 1.4559 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_gesture_2021-09-2915_48_41.305220/model-00002-1.60738-0.27731-1.45594-0.25000.h5
Epoch 3/30
21/21 [==============================] - 33s 2s/step - loss: 1.4883 - categorical_accuracy: 0.3669 - val_loss: 1.1959 - val_categorical_accuracy: 0.6250

Epoch 00003: saving model to model_gesture_2021-09-2915_48_41.305220/model-00003-1.48828-0.36695-1.19594-0.62500.h5
Epoch 4/30
21/21 [==============================] - 32s 2s/step - loss: 1.3836 - categorical_accuracy: 0.3978 - val_loss: 1.2160 - val_categorical_accuracy: 

##Train Accuracy: 98.04%
##Validation Accuracy: 93.75%

##Train accuracy has increased, and overfitting has also reduced. Thus, CNN+GRU with MobileNet as Transfer Learning model has given the best results.

END OF CODE